In [1]:
from qiskit import *
from qiskit.providers.fake_provider import FakeManila, FakeKolkata
from qiskit_ibm_provider import IBMProvider

import mthree
from mthree.utils import *
from mthree.generators.random import RandomGenerator, RandomComplimentGenerator
from mthree.generators import HadamardGenerator
from mthree._helpers import system_info

from mthree.twirling.tw_calibrations import Tw_Calibration
from mthree.twirling.tw_circuits import Tw_Circuits
from mthree.twirling.tw_utils import flatten_with_sum, minimal_qubits_to_calib, tw_expval

from qiskit.primitives import BackendEstimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

In [23]:
provider = IBMProvider()
#backend = FakeKolkata()
backend = provider.get_backend('ibm_seattle')
#len(backend.properties().qubits)

In [28]:
def GHZ(N):
    qc = QuantumCircuit(N, N)
    qc.h(0)
    for i in range(1,N):
        qc.cx(0,i)
    for i in range(N):
        qc.measure(i,i)
    trans_qc = transpile(qc, backend, optimization_level=2)
    return trans_qc

def m3circuit(N):
    qc = QuantumCircuit(N, N)
    qc.x(range(N))
    qc.h(range(N))

    for kk in range(N // 2, 0, -1):
        qc.ch(kk, kk - 1)
    for kk in range(N // 2, N - 1):
        qc.ch(kk, kk + 1)
    for i in range(N):
        qc.measure(i,i)
    trans_qc = transpile(qc, backend, optimization_level=2)
    return trans_qc

def donothing(N):
    qc = QuantumCircuit(N, N)
    for i in range(N):
        qc.measure(i,i)
    trans_qc = transpile(qc, backend, optimization_level=2)
    return trans_qc

def untranspiled_GHZ(N):
    qc = QuantumCircuit(N, N)
    qc.h(0)
    for i in range(1,N):
        qc.cx(0,i)
    for i in range(N):
        qc.measure(i,i)
    return qc

def untranspiled_m3circuit(N):
    qc = QuantumCircuit(N, N)
    qc.x(range(N))
    qc.h(range(N))

    for kk in range(N // 2, 0, -1):
        qc.ch(kk, kk - 1)
    for kk in range(N // 2, N - 1):
        qc.ch(kk, kk + 1)
    for i in range(N):
        qc.measure(i,i)
    return qc

def untranspiled_donothing(N):
    qc = QuantumCircuit(N, N)
    for i in range(N):
        qc.measure(i,i)
    return qc

In [29]:
repeats = 10
N_arr = [10]
unrepeated_circuit_list = [donothing(N) for N in N_arr]
unrepeated_op_list = [N*'Z' for N in N_arr]

circuit_list = []
for circuit in unrepeated_circuit_list:
    for _ in range(repeats):
        circuit_list.append(circuit)

op_list = []
for op in unrepeated_op_list:
    for _ in range(repeats):
        op_list.append(op)

qubits_to_calibrate = minimal_qubits_to_calib(circuit_list)
print(qubits_to_calibrate)

def all_z_op_string(qc, num_backend_qubits):
    operator_string = list(num_backend_qubits*'I')
    for key,value in final_measurement_mapping(qc).items():
        little_endian_index = num_backend_qubits-1-value
        operator_string[little_endian_index] = 'Z'
    operator_string = "".join(operator_string)
    return operator_string

def all_z_op(qc, num_backend_qubits):
    operator_string = list(num_backend_qubits*'I')
    for key,value in final_measurement_mapping(qc).items():
        little_endian_index = num_backend_qubits-1-value
        operator_string[little_endian_index] = 'Z'
    operator_string = "".join(operator_string)
    op = SparsePauliOp.from_list([(operator_string, 1)])
    return op

[160, 161, 195, 285, 134, 6, 338, 405, 282, 125]


In [30]:
circ_shots = 64
calib_shots = 512

tw_qc = Tw_Circuits(backend, circuits=circuit_list, generator=HadamardGenerator)
tw_qc.tw_data_from_backend(shots=circ_shots)

tw_cal = Tw_Calibration(backend, qubits=qubits_to_calibrate, generator=HadamardGenerator)
tw_cal.tw_calibrate_from_backend(shots=calib_shots)

results = []
for qc,op in zip(unrepeated_circuit_list, unrepeated_op_list):
    results.append(tw_expval(quantum_circ=qc, operator=op, tw_circuits=tw_qc, tw_calibration=tw_cal))

print(results)
     
#m3circuit_seattle_4_8_12_16 = [[0.3662109375, 0.4333694474539545], [0.22412109375, 0.3811500934191405], [0.1015625, 0.18842713169516476], [-0.0150146484375, -0.04507971412864211]]

In [ ]:
#print(results[0])
def extract_element(loc, arr):
    # extract all elements at location loc from an array of arrays 
    return np.array([sub_arr[loc] if len(sub_arr) >= loc+1 else None for sub_arr in arr])

mitigated_vals = extract_element(loc=1, arr=results[0])
print([np.average(mitigated_vals), 2*np.std(mitigated_vals)])

uncertainity_estimates = extract_element(loc=2, arr=results[0])
print(uncertainity_estimates[0])

[0.8446487536419552, 0.06104991827608264]
0.17357070227541066


In [ ]:
# # Make a noise model
# backend = FakeKolkata()
# noise_model = NoiseModel.from_backend(backend)
# num_backend_qubits = system_info(backend)["num_qubits"]

# # Set options to include the noise model
# options = Options()
# options.simulator = {
#     "noise_model": noise_model,
#     "basis_gates": backend.configuration().basis_gates,
#     "coupling_map": backend.configuration().coupling_map,
#     "seed_simulator": 42
# }

# # Set number of shots, optimization_level and resilience_level
# options.execution.shots = 1024
# options.optimization_level = 0
# options.resilience_level = 0

# # Set options to include the noise model with error mitigation
# options_with_em = Options()
# options_with_em.simulator = {
#     "noise_model": noise_model,
#     "basis_gates": backend.configuration().basis_gates,
#     "coupling_map": backend.configuration().coupling_map,
#     "seed_simulator": 42
# }

# # Set number of shots, optimization_level and resilience_level
# options_with_em.execution.shots = 1024
# options_with_em.optimization_level = 0 # no optimization
# options_with_em.resilience_level = 1 # M3 for Sampler and T-REx for Estimator

# qc = unrepeated_circuit_list[0]
# op = all_z_op(qc=qc, num_backend_qubits=num_backend_qubits)
# all_z_op_list = repeats*[op]

# with Session(service = QiskitRuntimeService(channel="ibm_quantum"), backend="ibmq_qasm_simulator"):
#     # # include the noise model without T-REx
#     # estimator = Estimator(options=options)
#     # #job = estimator.run(circuits=[qc], observables=[op])
#     # job = estimator.run(circuits=circuit_list, observables=all_z_op_list)
#     # result = job.result()
#     # print(f"Operator: {op}")
#     # print(f"Expectation value without T-REx: {np.average(result.values)}")
#     # print(f"Std without T-REx: {2*np.std(result.values)}")
#     # print(f"Metadata without T-REx: {result.metadata}")

#     # include the noise model with T-REx
#     estimator = Estimator(options=options_with_em)
#     #job = estimator.run(circuits=[qc], observables=[op])
#     job = estimator.run(circuits=circuit_list, observables=all_z_op_list)
#     result = job.result()
#     print(f"Operator: {op}")
#     print(f"Expectation value with T-REx: {np.average(result.values)}")
#     print(f"Std with T-REx: {2*np.std(result.values)}")
#     print(f"Metadata with T-REx: {result.metadata}")

In [ ]:
backend = provider.get_backend('ibm_seattle')
num_backend_qubits = system_info(backend)["num_qubits"]

# Set options to include the noise model
options = Options()

# Set number of shots, optimization_level and resilience_level
options.execution.shots = 1024
options.optimization_level = 0
options.resilience_level = 0

# Set options to include the noise model with error mitigation
options_with_em = Options()

# Set number of shots, optimization_level and resilience_level
options_with_em.execution.shots = 1024
options_with_em.optimization_level = 0 # no optimization
options_with_em.resilience_level = 1 # M3 for Sampler and T-REx for Estimator

qc = unrepeated_circuit_list[0]
op = all_z_op(qc=qc, num_backend_qubits=num_backend_qubits)
all_z_op_list = repeats*[op]

with Session(service = QiskitRuntimeService(channel="ibm_quantum"), backend='ibm_seattle'):
    # # include the noise model without T-REx
    # estimator = Estimator(options=options)
    # #job = estimator.run(circuits=[qc], observables=[op])
    # job = estimator.run(circuits=circuit_list, observables=all_z_op_list)
    # result = job.result()
    # print(f"Operator: {op}")
    # print(f"Avg Expectation value without T-REx: {np.average(result.values)}")
    # print(f"Std without T-REx: {2*np.std(result.values)}")
    # print(f"Metadata without T-REx: {result.metadata}")

    # include the noise model with T-REx
    estimator = Estimator(options=options_with_em)
    #job = estimator.run(circuits=[qc], observables=[op])
    job = estimator.run(circuits=circuit_list, observables=all_z_op_list)
    result = job.result()
    print(f"Operator: {op}")
    print(f"Expectation values with T-REx: {result.values}")
    print(f"Avg Expectation value with T-REx: {np.average(result.values)}")
    print(f"Std with T-REx: {2*np.std(result.values)}")
    print(f"Metadata with T-REx: {result.metadata}")

In [ ]:
# estimator = BackendEstimator(backend=backend,skip_transpilation=False)
# job = estimator.run(circuits=[qc], observables=[op])
# result = job.result()
# print(f"Operator: {op}")
# print(f"Expectation value: {result.values[0]}")
# print(f"Metadata: {result.metadata[0]}")
# # Run for a list of circuits and obtain the estimates for circuits on real hardware
# # implement code that does trex inside the runtime envrionment
# # see if it possible to reproduce the plot shared by Paul